In [457]:
import pandas as pd

In [458]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
py.init_notebook_mode(connected=True)

In [459]:
import requests, bs4
url = 'https://www.alltrails.com/parks/us/montana/glacier-national-park'
r = requests.get(url)
soup = bs4.BeautifulSoup(r.text, 'lxml')

In [460]:
ranking = []
link = []
name = []
diff = []
rating = []
location = []
distance = []
elevation = []
route_type =[]

In [461]:
for i in range(len(soup.find_all('div', class_="trail-result-card"))):
    
    # ranking
    try:
        ranking.append(soup.find_all('span',class_='item-rank xlate-none')[i].get_text())
    except Exception:
        ranking.append("")
   
    # link
    try:
        url = "https://www.alltrails.com" + (soup.find_all('a',class_='item-link')[i]['href'])
        link.append(url)
    except Exception:
        link.append("")
    
    # trail name
    try:
        name.append(soup.find_all('h3',class_='name xlate-none short')[i].get_text())
    except Exception:
        name.append("")
    
    # difficulty
    try:
        diff.append(soup.find_all('div',class_='difficulty-info')[i].get_text())
    except Exception:
        diff.append("")
    
    # rating
    try:
        rating.append(soup.find_all('div',class_='difficulty-info')[i].find_all('span')[1]['title'])
    except Exception:
        rating.append("")

    # location
    try:
        location.append(soup.find_all('div',class_='clickable xlate-none')[i]['title'])
    except Exception:
        location.append("")
    
    # GRAB TRAIL DETAILS
    r2 = requests.get(url)
    soup2 = bs4.BeautifulSoup(r2.text, 'lxml')
    
    # distance
    try:
        distance.append(soup2.find_all('div', class_="detail-data")[0].get_text())
    except Exception:
        distance.append("")
    
    # elevation gain
    try:
        elevation.append(soup2.find_all('div', class_="detail-data")[1].get_text())
    except Exception:
        elevation.append("")
    
    # route type
    try:
        route_type.append(soup2.find_all('div', class_="detail-data")[2].get_text())
    except Exception:
        route_type.append("")

In [462]:
df = pd.DataFrame(
    {'ranking': ranking,
     'num_reviews': num_reviews,
     'name': name,
     'difficulty': diff,
     'distance_miles': distance,
     'elevation_feet': elevation,
     'route_type': route_type,
     'rating': rating,
     'location': location,
     'link': link
    })

In [463]:
# format columns
df.num_reviews = df.num_reviews.str.strip("()").astype(int)
df.distance_miles = df.distance_miles.str.strip("miles").astype(float)
df.elevation_feet = df.elevation_feet.str.replace(",","").str.strip('feet').astype(int)
df.difficulty = df.difficulty.str.replace('[^a-zA-Z]','')
df['duration_hours'] = df.distance_miles/2
df['elevation_per_mile'] = df.elevation_feet/df.distance_miles

In [464]:
df = df[['ranking', 'num_reviews', 'name', 'difficulty','duration_hours','elevation_per_mile',
         'distance_miles','elevation_feet','route_type', 'rating', 'location', 'link']]

In [465]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

df_links = df.style.format({'link':make_clickable})

In [469]:
columns = ['num_reviews','duration_hours','elevation_per_mile','distance_miles','elevation_feet','rating']
for col in columns:
    t1 = go.Bar(x= df[df.difficulty == 'EASY']['ranking'],
                y= df[df.difficulty == 'EASY'][col],
                text=df[df.difficulty == 'EASY']['name'],
                textposition = 'auto',
                name='EASY')

    t2 = go.Bar(x= df[df.difficulty == 'MODERATE']['ranking'],
                y= df[df.difficulty == 'MODERATE'][col],
                text=df[df.difficulty == 'MODERATE']['name'],
                textposition = 'auto',
                name='MODERATE')

    t3 = go.Bar(x= df[df.difficulty == 'HARD']['ranking'],
                y= df[df.difficulty == 'HARD'][col],
                text=df[df.difficulty == 'HARD']['name'],
                textposition = 'auto',
                name='HARD')
    data = [t1,t2,t3]
    layout = go.Layout(title=col)
    fig = go.Figure(data,layout)
    py.iplot(fig)

SyntaxError: 'return' outside function (<ipython-input-469-687645e2ce95>, line 23)

In [468]:
df

,ranking,num_reviews,name,difficulty,duration_hours,elevation_per_mile,distance_miles,elevation_feet,route_type,rating,location,link
0,#1,926,Avalanche Lake via the Trail of the Cedars,MODERATE,2.85,131.228070,5.7,748,Out & Back,4.5,"Essex, Montana",https://www.alltrails.com/trail/us/montana/ava...
1,#2,690,Grinnell Glacier Trail,MODERATE,4.85,210.309278,9.7,2040,Out & Back,5,"Browning, Montana",https://www.alltrails.com/trail/us/montana/gri...
2,#3,835,Iceberg Lake Trail,MODERATE,4.65,155.913978,9.3,1450,Out & Back,5,"Browning, Montana",https://www.alltrails.com/trail/us/montana/ice...
3,#4,480,Hidden Lake Trail,MODERATE,2.65,259.245283,5.3,1374,Out & Back,4.5,"East Glacier Park Village, Montana",https://www.alltrails.com/trail/us/montana/hid...
4,#5,304,Highline Trail,HARD,14.20,172.359155,28.4,4895,Out & Back,5,"Browning, Montana",https://www.alltrails.com/trail/us/montana/hig...
5,#6,361,St. Mary and Virginia Falls Trail,EASY,1.45,155.862069,2.9,452,Out & Back,4.5,"East Glacier Park Village, Montana",https://www.alltrails.com/trail/us/montana/st-...
6,#7,384,Cracker Lake Trail,MODERATE,6.00,137.500000,12.0,1650,Out & Back,4.5,"Browning, Montana",https://www.alltrails.com/trail/us/montana/cra...
7,#8,276,Hidden Lake Overlook,MODERATE,1.40,207.142857,2.8,580,Out & Back,4.5,"East Glacier Park Village, Montana",https://www.alltrails.com/trail/us/montana/hid...
8,#9,288,The Garden Wall,HARD,7.35,238.571429,14.7,3507,Out & Back,5,"West Glacier, Montana",https://www.alltrails.com/trail/us/montana/the...
9,#10,239,Grinnell Lake Trail,EASY,3.55,49.436620,7.1,351,Loop,4.5,"Browning, Montana",https://www.alltrails.com/trail/us/montana/gri...
